<a href="https://colab.research.google.com/github/Bag0niku/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup
Install and import the dependencies needed to import and manipulate the data for Analysis.

In [1]:
# Install Spark and Java
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
import os
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# the SQL database we wil be using is Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

!pip install -q findspark # dependency to find the pyspark installation
import findspark  
findspark.init()  # initialize the search for pyspark

# Import remaining dependencies
from pyspark import SparkFiles
from pyspark.sql import SparkSession
import pyspark.sql
from getpass import getpass

# database password
print("What is the database password?")
password = getpass()


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://archive.ubuntu.com/ubuntu bionic-upda

In [2]:
# start Spark 
spark = SparkSession.builder.appName("AmazonReviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazon-vine-analysis.comqyjfkggk2.us-west-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}


# Deliverable 2: Determine Bias of Vine member reviews
Is any bias towards paid reviews that were written as part of the Vine program?  

In [3]:
# pull the vine table from the SQL database on AWS for this analysis
vine_table = spark.read.jdbc(url=jdbc_url, table="vine_table", properties=config)
vine_table.createOrReplaceTempView("vine_table")

# display the row count and the first 10 rows
spark.sql("SELECT COUNT(*) AS row_count FROM vine_table;").show()
vine_table.show(10)


+---------+
|row_count|
+---------+
|  1785997|
+---------+

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R12QUNRNIZPT5E|          1|            3|          4|   N|                Y|
| RCQW57TVTB1Y0|          4|            0|          0|   N|                Y|
|R29JGO9JQO83P4|          1|            0|          2|   N|                Y|
|R3LKGEEFM2ZICN|          3|            1|          1|   N|                Y|
| RMYHLWMHF5FJX|          4|            0|          0|   N|                N|
| RPKHUN8LMZA57|          5|            0|          0|   N|                Y|
| RH3MQLUMCGC8U|          4|            0|          0|   N|                Y|
|R3EWF1W0LPO06T|          5|            0|          0|   N|                Y|
|R31OOHO4IW5BBM|          3|            0|          0|   N|                Y|
|R3

In [4]:
# import the review_id_table to join the analysis
review_id_table = spark.read.jdbc(url=jdbc_url, table="review_id_table", properties=config)
review_id_table.createOrReplaceTempView("review_id_table")

# display the row count and the first 10 rows
spark.sql("SELECT COUNT(*) AS row_count FROM review_id_table;").show()
review_id_table.show(10)

+---------+
|row_count|
+---------+
|  1785997|
+---------+

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RMNPN9O01CP4R|   15404454|B0085CA8NY|     536556164| 2014-10-24|
| RL2KMRJV3POPV|   38780073|B002OL1KEI|     320413976| 2014-10-24|
| RJ8XPG45SBLLV|   32924447|B00GXKV8S4|     441811350| 2014-10-24|
| RA9P12Q1GW2F3|   13258963|B002I096Q4|      15859536| 2014-10-24|
| RNQI9WLCSMI17|   14120968|B0050SYILE|     290750095| 2014-10-24|
|R3OPW5AN28TW3M|   12650972|B00JKM09Z0|     173798294| 2014-10-24|
|R3KE0ZDEMCEPV4|   32924447|B00ECOAX9I|     202522284| 2014-10-24|
| R9GE34IN2TKCG|   10066660|B00GT8FUZC|      90147553| 2014-10-24|
|R10L6DNHF9D0OV|   35837452|B00DHF3A4S|      39203705| 2014-10-24|
|R312A29FX9YU7P|   18305278|B000FQ2DTA|     903003386| 2014-10-24|
+--------------+-----------+----------+--------------+-----------+
o

In [6]:
# join the review_id table and the vine_table for viewing the impact of paid reviews on products
products_reviews_table = spark.sql("""SELECT vine.*, review.product_id
             FROM vine_table AS vine
             JOIN review_id_table AS review ON review.review_id == vine.review_id;""")
products_reviews_table.createOrReplaceTempView("products_reviews_table")

# display the row count and the first 10 rows
spark.sql("SELECT COUNT(*) AS row_count FROM products_reviews_table;").show()
products_reviews_table.show(10)

+---------+
|row_count|
+---------+
|  1785997|
+---------+

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|product_id|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R10003I619LWL0|          5|            1|          1|   N|                N|B001REZLY8|
|R100078OO83YQB|          5|            0|          0|   N|                N|B0016HM45K|
|R10009XH2FY9Q0|          5|            4|          4|   N|                N|B00000IOQV|
|R1000EJULTHQ16|          5|            0|          0|   N|                N|B002I0H7K6|
|R1000GECIM9DZG|          5|            0|          0|   N|                Y|B006SVUQNQ|
|R1000ID99V4TKO|          5|            0|          0|   N|                N|B00KY1HZ80|
|R1000U9GM6NPGS|          5|            0|          0|   N|                Y|B00DUJKIO6|
|R1000WZPINB01L|          4|            0|       

In [16]:
# create a table with products that have enough votes to run an analysis.
# a minimum 20 total votes for any review.
usable_vine_table = spark.sql("SELECT * FROM products_reviews_table WHERE total_votes > 19;")
usable_vine_table.createOrReplaceTempView("usable_vine_table")

# display the row count and the first 10 rows
spark.sql("SELECT COUNT(*) AS row_count FROM usable_vine_table;").show()
usable_vine_table.show(10)

+---------+
|row_count|
+---------+
|    65379|
+---------+

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|product_id|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R2RQBEFRD5W6HP|          1|           10|         24|   N|                Y|B00CJXYTGM|
|R2MT86YP52BQ4N|          1|            8|         20|   N|                N|B00BGA9WK2|
|R3EFO5L1DNBXXB|          1|           16|         22|   N|                N|B00BGA9Y3W|
|R1IMG9MN73019I|          5|           55|         72|   N|                N|B00DHF39HQ|
| RIUS6OCG18T5J|          1|            3|         41|   N|                N|B002I096Q4|
|R2Y5IU07L3YSC8|          1|            7|         48|   N|                N|B00DHF39HQ|
|R3HYYDI3RWONZI|          1|            6|         28|   N|                N|B00TS0UTAY|
|R3G4E4WCZ34G6A|          1|            4|       

In [17]:
# filter usable_vine_table for where helpful votes is atleast 50% of the total votes
helpful_votes_table = spark.sql("SELECT * FROM usable_vine_table WHERE ((helpful_votes/total_votes) >= 0.5);")
helpful_votes_table.createOrReplaceTempView("helpful_votes_table")

# display the row count and the first 10 rows
spark.sql("select count(*) as row_count from helpful_votes_table;").show()
helpful_votes_table.show(10)

+---------+
|row_count|
+---------+
|    40565|
+---------+

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|product_id|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R3EFO5L1DNBXXB|          1|           16|         22|   N|                N|B00BGA9Y3W|
|R1IMG9MN73019I|          5|           55|         72|   N|                N|B00DHF39HQ|
|R1AYAL8OR1RXT9|          5|           33|         37|   N|                Y|B00KSY5486|
| R4759EBMM5S30|          1|           19|         34|   N|                N|B00CX8VY4S|
|R3865R90FXCLT1|          1|          192|        233|   N|                Y|B00EM5UFEK|
|R3K0D1MAF71A3Z|          2|           14|         22|   N|                Y|B00DBLBMBQ|
|R2JDVD6YCNSWNB|          2|           30|         36|   N|                Y|B003NSLGW2|
|R1GQ534F21DRP0|          1|           12|       

In [30]:
# filter usable_vine_reviews for only vine member reviews
vine_reviews_table = spark.sql("SELECT * FROM usable_vine_table WHERE vine == 'Y' ;")
vine_reviews_table.createOrReplaceTempView("vine_reviews_table")

# display the row count and the first 10 rows
spark.sql("SELECT COUNT(*) AS row_count FROM vine_reviews_table;").show()
vine_reviews_table.show(30)

+---------+
|row_count|
+---------+
|      104|
+---------+

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|product_id|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
| R61QRS16PUHET|          5|           38|         41|   Y|                N|B0058SHMXM|
|R24PS9N276OXF4|          5|          241|        262|   Y|                N|B00KSQAACO|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|B00O9GVKZU|
|R3NGUZ07DTUUSY|          3|           53|         62|   Y|                N|B008J16AG0|
|R2GYO7WKCP4TOM|          3|           18|         25|   Y|                N|B0086V5V9A|
|R15ARADYJE29FK|          3|           33|         36|   Y|                N|B0072A4GQK|
| R4AMWJFZ9W2LN|          4|            6|         33|   Y|                N|B0086V5V8G|
| R8YT75NJW0CM9|          4|           37|       

In [31]:
# filter usable reviews for only non-vine member reviews
not_member_reviews_table = spark.sql("SELECT * FROM usable_vine_table WHERE vine =='N';")
not_member_reviews_table.createOrReplaceTempView("not_member_reviews_table")
spark.sql("SELECT COUNT(*) AS row_count FROM not_member_reviews_table;").show()
not_member_reviews_table.show(30)

+---------+
|row_count|
+---------+
|    65275|
+---------+

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|product_id|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R3HYYDI3RWONZI|          1|            6|         28|   N|                N|B00TS0UTAY|
|R3G4E4WCZ34G6A|          1|            4|         30|   N|                N|B00S8IGG4U|
|R1AYAL8OR1RXT9|          5|           33|         37|   N|                Y|B00KSY5486|
|R3AG1EEH1SP6XH|          1|           10|         24|   N|                N|B00VM27UVW|
| R3EF736S5J90L|          1|            8|         29|   N|                N|B00KWG4HG0|
| R4759EBMM5S30|          1|           19|         34|   N|                N|B00CX8VY4S|
|R3865R90FXCLT1|          1|          192|        233|   N|                Y|B00EM5UFEK|
|R3K0D1MAF71A3Z|          2|           14|       

# Summary Tables: 
Below are the summary tables created from the categorized data above.

In [26]:
# total summary table
spark.sql("""SELECT (SELECT COUNT(*) FROM products_reviews_table) AS total_reviews, 
                    (SELECT COUNT(*) FROM products_reviews_table WHERE star_rating == 5) AS total_5_star_reviews,
                    (SELECT COUNT(*) FROM products_reviews_table WHERE (star_rating == 5) AND (vine == 'Y')) AS vine_total_5_star,
                    ROUND(((SELECT COUNT(*) FROM products_reviews_table WHERE (star_rating == 5) AND (vine == 'Y'))/(SELECT COUNT(*) FROM products_reviews_table WHERE star_rating == 5))*100,2 ) AS vine_percent_5_star,
                    (SELECT COUNT(*) FROM products_reviews_table WHERE (star_rating == 5) AND (vine == 'N')) AS not_vine_total_5_star,
                    ROUND(((SELECT COUNT(*) FROM products_reviews_table WHERE (star_rating == 5) AND (vine == 'N'))/(SELECT COUNT(*) FROM products_reviews_table WHERE star_rating == 5))*100, 2) AS not_vine_percent_5_star;""").show()


+-------------+--------------------+-----------------+-------------------+---------------------+-----------------------+
|total_reviews|total_5_star_reviews|vine_total_5_star|vine_percent_5_star|not_vine_total_5_star|not_vine_percent_5_star|
+-------------+--------------------+-----------------+-------------------+---------------------+-----------------------+
|      1785997|             1026924|             1607|               0.16|              1025317|                  99.84|
+-------------+--------------------+-----------------+-------------------+---------------------+-----------------------+



In [20]:
# usable reviews summary table
spark.sql("""SELECT (SELECT COUNT(*) FROM usable_vine_table) AS usable_reviews, 
                    (SELECT COUNT(*) FROM usable_vine_table WHERE star_rating == 5) AS total_usable_5_star_reviews,
                    (SELECT COUNT(*) FROM vine_reviews_table WHERE (star_rating == 5)) AS vine_total_usable_5_star,
                    ROUND(((SELECT COUNT(*) FROM vine_reviews_table WHERE (star_rating == 5))/(SELECT COUNT(*) FROM usable_vine_table WHERE star_rating == 5))*100, 2) AS vine_percent_5_star,
                    (SELECT COUNT(*) FROM not_member_reviews_table WHERE (star_rating == 5)) AS not_vine_total_usable_5_star,
                    ROUND(((SELECT COUNT(*) FROM not_member_reviews_table WHERE (star_rating == 5))/(SELECT COUNT(*) FROM usable_vine_table WHERE (star_rating == 5)))*100, 2) AS not_vine_percent_5_star;""").show()

+--------------+---------------------------+------------------------+-------------------+----------------------------+-----------------------+
|usable_reviews|total_usable_5_star_reviews|vine_total_usable_5_star|vine_percent_5_star|not_vine_total_usable_5_star|not_vine_percent_5_star|
+--------------+---------------------------+------------------------+-------------------+----------------------------+-----------------------+
|         65379|                      20487|                      48|               0.23|                       20439|                  99.77|
+--------------+---------------------------+------------------------+-------------------+----------------------------+-----------------------+



In [23]:
# summary table of vine member reviews that recieved atleast 1 helpful vote and very helpful > = 50%+ helpful/total
spark.sql("""SELECT (SELECT COUNT(*) FROM products_reviews_table WHERE helpful_votes >0) AS count_helpful_reviews,
                    (SELECT COUNT(*) FROM vine_reviews_table WHERE (helpful_votes > 0)) AS vine_count_helpful, 
                    (SELECT COUNT(*) FROM vine_reviews_table WHERE (helpful_votes >0) AND (helpful_votes/total_votes >= 0.5)) AS vine_count_very_helpful,
                    (SELECT COUNT(*) FROM vine_reviews_table WHERE (helpful_votes >0) AND (star_rating == 5)) AS vine_count_helpful_5_star;""").show()

+---------------------+------------------+-----------------------+-------------------------+
|count_helpful_reviews|vine_count_helpful|vine_count_very_helpful|vine_count_helpful_5_star|
+---------------------+------------------+-----------------------+-------------------------+
|               698326|               102|                     94|                       48|
+---------------------+------------------+-----------------------+-------------------------+



In [27]:
# summary table of not vine member reviews that recieved atleast 1 helpful vote and very helpful > = 50%+ helpful/total
spark.sql("""SELECT (SELECT COUNT(*) FROM products_reviews_table WHERE helpful_votes >0) AS count_helpful_reviews,
                    (SELECT COUNT(*) FROM not_member_reviews_table WHERE (helpful_votes > 0)) AS not_vine_count_helpful,
                    (SELECT COUNT(*) FROM not_member_reviews_table WHERE (helpful_votes >0) AND (helpful_votes/total_votes >= 0.5)) AS not_vine_count_very_helpful,
                    (SELECT COUNT(*) FROM not_member_reviews_table WHERE (helpful_votes >0) AND (star_rating == 5)) AS not_vine_count_helpful_5_star;""").show()

+---------------------+----------------------+---------------------------+-----------------------------+
|count_helpful_reviews|not_vine_count_helpful|not_vine_count_very_helpful|not_vine_count_helpful_5_star|
+---------------------+----------------------+---------------------------+-----------------------------+
|               698326|                 64008|                      40471|                        20282|
+---------------------+----------------------+---------------------------+-----------------------------+



In [29]:
# summary table counting distinct products with reviews not reviewed by opposite category vine or not vine
spark.sql("""SELECT (SELECT COUNT(DISTINCT(product_id)) FROM not_member_reviews_table WHERE product_id not in (SELECT product_id FROM vine_reviews_table)) AS products_without_vine_reviews,
                    (SELECT COUNT(DISTINCT(product_id)) FROM VINE_reviews_table WHERE product_id not in (SELECT product_id FROM not_member_reviews_table)) AS products_with_only_vine_reviews,
                    (SELECT COUNT(DISTINCT(product_id)) FROM VINE_reviews_table WHERE product_id in (SELECT product_id FROM not_member_reviews_table)) AS products_with_both_reviews;""").show()

+-----------------------------+-------------------------------+--------------------------+
|products_without_vine_reviews|products_with_only_vine_reviews|products_with_both_reviews|
+-----------------------------+-------------------------------+--------------------------+
|                        14249|                             19|                        51|
+-----------------------------+-------------------------------+--------------------------+



# Results: Paid vs Unpaid

- There are significantly more reviews by non-vine members than reviews by vine members.  
- Of the 14319 products in the dataset
    - 14249 products have no paid reviews
    - 19 products have only paid reviews
    - 51 products have reviews from both categories.

 - 1785997 reviews in total
     - 1026924 of those reviews are 5 stars: 99.84% are unpaid,  0.16% are paid
     - 69k+ reviews were voted helpful, heavily favoring the unpaid reviews (99%+)


There is no bias towards paid reviews.
